In [ ]:
!pip install MIDIUtil

In [ ]:
!pip install dawdreamer

In [ ]:
!pip install mido

In [ ]:
import pandas as pd
file_path = "/content/drive/MyDrive/global-temp.csv"
df = pd.read_csv(file_path)
data_info = df.info()
data_head = df.head()
missing_values = df.isnull().sum()
outlier_summary = df.describe()
data_info, data_head, missing_values, outlier_summary

In [ ]:
df

In [ ]:
from midiutil import MIDIFile
import numpy as np

# Function to map temperature to MIDI note numbers
def temp_to_midi(temp, temp_min, temp_max, note_min=60, note_max=80):
    return int(np.interp(temp, [temp_min, temp_max], [note_min, note_max]))

# Create a MIDI file with one track
midi = MIDIFile(1)
track = 0
time = 0  # Start at the beginning
midi.addTrackName(track, time, "Climate Data Music")
midi.addTempo(track, time, 120)

# Use only the first 20 rows of the dataset
df_subset = df.head(20)

# Normalize temperature values to MIDI notes (LandAverageTemperature column)
min_temp = df_subset['LandAverageTemperature'].min()
max_temp = df_subset['LandAverageTemperature'].max()

for i, row in df_subset.iterrows():
    temperature = row['LandAverageTemperature']
    note = temp_to_midi(temperature, min_temp, max_temp)

    # Add note to the MIDI file (track, channel, pitch, time, duration, volume)
    midi.addNote(track, 0, note, time, 1, 100)  # 1 second duration, volume 100
    time += 1  # Move to the next time step

# Write the MIDI file to disk
midi_file_path = "/content/drive/MyDrive/climate_data_music_first_20_rows.mid"
with open(midi_file_path, "wb") as output_file:
    midi.writeFile(output_file)

midi_file_path  # Return the path to the MIDI file


In [ ]:
import random
import numpy as np
from midiutil import MIDIFile

In [ ]:
import mido
from mido import MidiFile, Message
import pandas as pd
import random

#for first row
first_row = df.iloc[0]

# Get parameter values
LandAverageTemperature = first_row['LandAverageTemperature']
LandAverageTemperatureUncertainty = first_row['LandAverageTemperatureUncertainty']
LandMaxTemperature = first_row['LandMaxTemperature']
LandMaxTemperatureUncertainty = first_row['LandMaxTemperatureUncertainty']
LandMinTemperature = first_row['LandMinTemperature']
LandMinTemperatureUncertainty = first_row['LandMinTemperatureUncertainty']
LandAndOceanAverageTemperature = first_row['LandAndOceanAverageTemperature']
LandAndOceanAverageTemperatureUncertainty = first_row['LandAndOceanAverageTemperatureUncertainty']

# Define chord progressions with nested lists
chord_progressions = {
    1: [[60, 64, 67], [67, 71, 74], [69, 72, 76], [65, 69, 72]],  # C-G-Am-F (I-V-vi-IV)
    2: [[60, 64, 67], [69, 72, 76], [62, 65, 69], [67, 71, 74]],  # C-Am-Dm-G (I-vi-ii-V)
    3: [[60, 64, 67], [65, 69, 72], [69, 72, 76], [67, 71, 74]],  # C-F-Am-G (I-IV-vi-V)
    4: [[60, 64, 67], [65, 69, 72], [67, 71, 74]],                  # C-F-G (I-IV-V)
    5: [[60, 64, 67], [67, 71, 74], [65, 69, 72], [69, 72, 76]],  # C-G-F-Am (I-V-IV-vi)
    6: [[62, 65, 69], [67, 71, 74], [60, 64, 67]],                  # Dm-G-C (ii-V-I)
}


In [ ]:
import mido
from mido import MidiFile, Message
import pandas as pd
import random

# Get parameter values
LandAverageTemperature = first_row['LandAverageTemperature']
LandAverageTemperatureUncertainty = first_row['LandAverageTemperatureUncertainty']

# Set parameters for the MIDI file
base_duration = 480  # Base duration in ticks
base_velocity = 64  # Base velocity
tempo = 120  # BPM
note_duration = 480  # Duration in ticks
repetitions = 4 # Number of times to repeat each chord progression

In [ ]:

# Create a new MIDI file
midi_file = MidiFile()
track = midi_file.add_track('Chords')
percussion_track = midi_file.add_track('Percussion')

# Function to add notes for a given chord
def add_chord_notes(chord, time_offset, duration, velocity):
    for note in chord:
        track.append(Message('note_on', note=note, velocity=velocity, time=time_offset))
    for note in chord:
        track.append(Message('note_off', note=note, velocity=velocity, time=duration))

# Function to add pleasant percussion beats that sync with chord duration
def add_percussion_beat(start_time, note_duration):

    percussion_pattern = [
        (80, start_time),
        (80, start_time + note_duration),
        (80, start_time + 2 * note_duration),
        (80, start_time + 3 * note_duration)
    ]

    for note, time in percussion_pattern:
        percussion_track.append(Message('note_on', note=note, velocity=100, time=time))
        percussion_track.append(Message('note_off', note=note, velocity=100, time=note_duration))

# Generate music based on parameters
def generate_music_from_row():
    for _ in range(4):  # Repeat progressions
        progression = random.choice(list(chord_progressions.values()))

        for chord in progression:
            # Adjust pitch based on temperature
            adjusted_chord = [note + int(LandAverageTemperature * 2) % 12 for note in chord]

            # Adjust duration and velocity
            note_duration = base_duration + int(LandAverageTemperatureUncertainty * 2)
            velocity = base_velocity + int(LandAverageTemperature * 1.5)

            # Ensure velocity is within MIDI limits (0-127)
            velocity = max(0, min(127, velocity))

            # Add chord notes
            add_chord_notes(adjusted_chord, 0, note_duration, velocity)
            # Add percussion beat, ensuring it only plays during the chord notes
            add_percussion_beat(0, note_duration)

# Generate music for the first row of dataset
generate_music_from_row()

# Save the MIDI file
midi_file.save('generated_music_with_percussion.mid')

In [ ]:
import mido

def truncate_midi(input_file, output_file, duration_seconds=30):
    # Load the MIDI file
    midi = mido.MidiFile(input_file)

    # Define parameters
    bpm = 120  # You can adjust this to your file's BPM
    ticks_per_beat = midi.ticks_per_beat
    duration_ticks = int(bpm * duration_seconds * ticks_per_beat / 60)

    # Create a new MIDI file for truncated output
    truncated_midi = mido.MidiFile()

    # Loop through the tracks
    for track in midi.tracks:
        new_track = mido.MidiTrack()
        time_counter = 0

        # Loop through the messages in the track
        for msg in track:
            # Add message time to the time counter
            time_counter += msg.time

            # Stop adding messages if we've reached the duration limit
            if time_counter >= duration_ticks:
                break

            new_track.append(msg)

        truncated_midi.tracks.append(new_track)

    # Save the truncated MIDI file
    truncated_midi.save(output_file)

# Example usage
truncate_midi('generated_music_with_percussion.mid', 'truncated_music.mid', duration_seconds=25)
